In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('resources/data.csv')
df_copy = df.loc[:, (df != 0).any(axis=0)]
df_copy = df_copy.drop(labels=['Sprint', 'Severity of the threat'], axis=1)

In [2]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516 entries, 0 to 515
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Change code line number                   516 non-null    int64 
 1   Number of vulnerable modules              516 non-null    int64 
 2   Number of people involved in development  516 non-null    int64 
 3   Time to complete each version             516 non-null    object
 4   Commit frequency                          516 non-null    object
 5   Type of environment                       516 non-null    object
 6   Number of libraries detected errors       516 non-null    int64 
 7   Scan date                                 516 non-null    object
 8   Number of potential weaknesses            516 non-null    int64 
 9   Số lượng lỗ hổng cấu hình môi trường      516 non-null    int64 
 10  Evaluate                                  516 non-

In [3]:
df_copy

,Change code line number,Number of vulnerable modules,Number of people involved in development,Time to complete each version,Commit frequency,Type of environment,Number of libraries detected errors,Scan date,Number of potential weaknesses,Số lượng lỗ hổng cấu hình môi trường,Evaluate,Repo
0,19,1,2,5 days,Daily,Cloud,22,2025-02-17 00:00:00,26,4,4,admin-jlpt
1,1152,1,2,5 days,Daily,Cloud,22,2025-02-18 00:00:00,26,4,4,admin-jlpt
2,100,1,2,5 days,Daily,Cloud,22,2025-02-19 00:00:00,26,4,4,admin-jlpt
3,221,1,2,5 days,Daily,Cloud,22,2025-02-20 00:00:00,26,4,4,admin-jlpt
4,96,1,2,5 days,Daily,Cloud,22,2025-02-22 00:00:00,26,4,4,admin-jlpt
...,...,...,...,...,...,...,...,...,...,...,...,...
511,530,1,8,5 days,Weekly,Cloud,40,2025-01-09 00:00:00,42,2,2,payment-system
512,1996,1,6,5 days,Daily,Cloud,39,2025-02-04 00:00:00,40,1,3,user-management
513,1986,1,8,7 days,Weekly,Cloud,14,2025-01-15 00:00:00,16,2,3,payment-system
514,435,1,7,10 days,Monthly,Cloud,29,2025-03-14 00:00:00,32,3,1,admin-jlpt


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

def preprocess_timeseries_dataframe(df: pd.DataFrame):
    df = df.copy()
    df = df.loc[:, (df != 0).any(axis=0)]
    df = df.drop(labels=['Sprint', 'Severity of the threat'], axis=1)
    df['Scan date'] = pd.to_datetime(df['Scan date'])

    categorical_columns = ['Time to complete each version', 'Commit frequency', 'Type of environment', 'Repo']
    group_sum_col = 'Change code line number'
    target_col = 'Evaluate'
    date_col = 'Scan date'

    all_dfs = []
    for repo, group in df.groupby('Repo'):
        # group by date to avoid duplicates before reindexing
        group = group.groupby(date_col).agg({
            group_sum_col: 'sum',
            'Number of vulnerable modules': 'mean',
            'Number of people involved in development': 'mean',
            'Number of libraries detected errors': 'mean',
            'Number of potential weaknesses': 'mean',
            'Số lượng lỗ hổng cấu hình môi trường': 'mean',
            target_col: 'first',
            **{col: 'first' for col in categorical_columns}
        }).sort_index()

        # create full date index and reindex
        all_dates = pd.date_range(start=group.index.min(), end=group.index.max(), freq='D')
        group = group.reindex(all_dates, method='ffill')
        group[date_col] = group.index
        group['Repo'] = repo

        # identify filled-in rows and set code line to 0
        original_dates = df[df['Repo'] == repo]['Scan date'].unique()
        group[group_sum_col] = group[group_sum_col].where(group.index.isin(original_dates), 0)

        all_dfs.append(group)

    df_filled = pd.concat(all_dfs).reset_index(drop=True)

    # encode categoricals
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        df_filled[col] = le.fit_transform(df_filled[col])
        label_encoders[col] = le

    # normalize features (excluding Scan date, Repo, Evaluate)
    feature_cols = df_filled.columns.difference([date_col, 'Repo', target_col])
    scaler = StandardScaler()
    df_filled[feature_cols] = scaler.fit_transform(df_filled[feature_cols])

    # final outputs
    X = df_filled.drop(columns=[date_col, 'Repo', target_col])
    y = df_filled[target_col]
    meta = df_filled[[date_col, 'Repo']]

    return X, y, meta, label_encoders, scaler

In [5]:
X, y, meta, encoders, scaler = preprocess_timeseries_dataframe(df)
X

,Change code line number,Number of vulnerable modules,Number of people involved in development,Number of libraries detected errors,Number of potential weaknesses,Số lượng lỗ hổng cấu hình môi trường,Time to complete each version,Commit frequency,Type of environment
0,1.078302,0.0,-0.649854,0.488260,0.552000,0.950836,-1.198149,0.096718,0.0
1,0.010589,0.0,-0.067334,-1.500259,-1.611377,-1.724206,-1.198149,-1.099074,0.0
2,-0.064960,0.0,-0.566637,-0.601339,-0.556731,0.568687,1.233915,-1.099074,0.0
3,0.430454,0.0,1.929876,0.706180,0.579042,-1.724206,0.017883,1.292511,0.0
4,0.766747,0.0,-0.316985,0.542740,0.538479,-0.004536,1.233915,0.096718,0.0
...,...,...,...,...,...,...,...,...,...
267,1.052897,0.0,0.431968,-0.002059,0.038198,0.568687,-1.198149,-1.099074,0.0
268,1.032839,0.0,-1.731676,-0.383419,-0.475604,-1.342057,1.233915,1.292511,0.0
269,0.007915,0.0,-1.565242,1.278220,1.146929,-1.724206,-1.198149,1.292511,0.0
270,-0.400584,0.0,0.931271,1.931980,1.958195,0.568687,-1.198149,1.292511,0.0


In [6]:
y

0      4
1      3
2      2
3      3
4      2
      ..
267    4
268    4
269    1
270    1
271    3
Name: Evaluate, Length: 272, dtype: int64

In [7]:
meta

,Scan date,Repo
0,2025-01-01,0
1,2025-01-02,0
2,2025-01-03,0
3,2025-01-04,0
4,2025-01-05,0
...,...,...
267,2025-03-27,2
268,2025-03-28,2
269,2025-03-29,2
270,2025-03-30,2


In [8]:
ori_X = scaler.inverse_transform(X[list(scaler.feature_names_in_)]).astype(int)
ori_X = pd.DataFrame(data=ori_X, columns=list(scaler.feature_names_in_))

In [9]:
for col in ('Time to complete each version', 'Commit frequency', 'Type of environment'):
    ori_X[col] = encoders[col].inverse_transform(ori_X[col])

In [10]:
ori_X

,Change code line number,Commit frequency,Number of libraries detected errors,Number of people involved in development,Number of potential weaknesses,Number of vulnerable modules,Số lượng lỗ hổng cấu hình môi trường,Time to complete each version,Type of environment
0,3494,Monthly,31,3,34,1,3,10 days,Cloud
1,1897,Daily,7,5,8,1,1,10 days,Cloud
2,1784,Daily,18,4,21,1,3,7 days,Cloud
3,2525,Weekly,34,9,35,1,1,5 days,Cloud
4,3028,Monthly,32,4,34,1,2,7 days,Cloud
...,...,...,...,...,...,...,...,...,...
267,3456,Daily,25,6,28,1,3,10 days,Cloud
268,3426,Weekly,20,1,22,1,1,7 days,Cloud
269,1893,Weekly,41,2,42,1,1,10 days,Cloud
270,1282,Weekly,49,7,52,1,3,10 days,Cloud


In [11]:
ori_meta  = meta.copy()
ori_meta['Repo'] = encoders['Repo'].inverse_transform(ori_meta['Repo'])
ori_meta

,Scan date,Repo
0,2025-01-01,admin-jlpt
1,2025-01-02,admin-jlpt
2,2025-01-03,admin-jlpt
3,2025-01-04,admin-jlpt
4,2025-01-05,admin-jlpt
...,...,...
267,2025-03-27,user-management
268,2025-03-28,user-management
269,2025-03-29,user-management
270,2025-03-30,user-management


In [12]:
ori_y = pd.DataFrame(data=y, columns=['Evaluate'])
ori_y

,Evaluate
0,4
1,3
2,2
3,3
4,2
...,...
267,4
268,4
269,1
270,1


In [13]:
col_names = list(ori_X.columns) + list(ori_meta.columns) + list(ori_y.columns)
ori_data = pd.concat([ori_X, ori_meta, ori_y], ignore_index=True, axis=1)
ori_data.columns = col_names
ori_data = ori_data[df_copy.columns.tolist()]
ori_data.to_csv('resources/data_cleaned.csv', index=False)

### Model

In [14]:
y = y - y.min()

In [15]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [34]:
def build_model(config_id):
    if config_id == 'XGB':
        return XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    elif config_id == 'RF':
        return RandomForestClassifier(n_estimators=200)
    elif config_id == 'SVM':
        return SVC(kernel='rbf', probability=True)
    elif config_id == 'MLP':
        return MLPClassifier(hidden_layer_sizes=(32, 32), max_iter=25000)
    elif config_id == 'LogReg':
        return LogisticRegression(max_iter=1000)
    elif config_id == 'KNN':
        return KNeighborsClassifier(n_neighbors=5)
    else:
        raise ValueError(f"Unknown config id: {config_id}")


def run_experiment(config_id, X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    model = build_model(config_id)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average='weighted', zero_division=0)

    print(f"[{config_id}] Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return {
        'config': config_id,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [35]:
all_configs = ['XGB', 'RF', 'SVM', 'MLP', 'LogReg', 'KNN']
results = [run_experiment(cfg, X.values, y.values) for cfg in all_configs]

/home/thanh309/miniconda3/envs/thanh309-ml/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [13:08:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[XGB] Accuracy: 0.3273, Precision: 0.3208, Recall: 0.3273, F1: 0.3199
[RF] Accuracy: 0.3636, Precision: 0.3529, Recall: 0.3636, F1: 0.3534
[SVM] Accuracy: 0.2727, Precision: 0.1623, Recall: 0.2727, F1: 0.1985
[MLP] Accuracy: 0.3091, Precision: 0.3040, Recall: 0.3091, F1: 0.3050
[LogReg] Accuracy: 0.3273, Precision: 0.2697, Recall: 0.3273, F1: 0.2587
[KNN] Accuracy: 0.3455, Precision: 0.3491, Recall: 0.3455, F1: 0.3343
